In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.utils import shuffle
from sklearn.metrics import auc, roc_curve, f1_score, average_precision_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold


import matplotlib.pyplot as plt
%matplotlib inline

## Load and prep exmple data

In [2]:
df = pd.read_csv("./data/creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
X = np.array(df.filter(regex="V.*"))
y = np.array(df.Class)

## Applying SMOTE and Variants

In [4]:
from imblearn.over_sampling import SMOTE

In [6]:
RANDOM_STATE = 42
samplers = [
    ['SMOTE_borderline1', SMOTE(kind = "borderline1", random_state=RANDOM_STATE)],
    ['SMOTE_borderline2', SMOTE(kind = "borderline2", random_state=RANDOM_STATE)],
    ['SMOTE_regular', SMOTE(random_state=RANDOM_STATE)]
]


In [10]:
%%time
cv = StratifiedKFold(n_splits=5, random_state=42)
baseline_mean_aucs = {}

for name, sampler in samplers:
    print("working on", name)
    aucs_train = []
    aucs = []
    for train, test in cv.split(X,y):
        clf = RandomForestClassifier(n_estimators=100, n_jobs=-2)
        X_resampled, y_resampled = sampler.fit_sample(X[train], y[train])
        clf.fit(X_resampled, y_resampled)
        y_scores = clf.predict_proba(X[test])[:,1]
        aucs.append(roc_auc_score(y[test], y_scores))
    
    
    baseline_mean_aucs[name] = aucs

working on SMOTE_borderline1
working on SMOTE_borderline2
working on SMOTE_regular
CPU times: user 2h 49min 55s, sys: 2.44 s, total: 2h 49min 58s
Wall time: 6min 28s


In [11]:
for key in baseline_mean_aucs.keys():
    print(key)
    print(np.mean(baseline_mean_aucs[key]), np.std(baseline_mean_aucs[key]) )

SMOTE_borderline1
0.950483268684 0.0233165151174
SMOTE_borderline2
0.95734826301 0.0244896557083
SMOTE_regular
0.968752713529 0.0160447056377


## Bagged Subset SMOTE: BASS!!

In [5]:
from bass import *
import warnings
warnings.filterwarnings("ignore", 
  message="Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.")
warnings.filterwarnings("ignore", message="From version 0.21, test_size \
  will always complement train_size unless both are specified")

In [21]:
RANDOM_STATE = 42
# samplers = [
#     ['SMOTE_borderline1', SubsetSMOTE(synth_features=.9, kind = "borderline1")],
#     ['SMOTE_borderline2', SubsetSMOTE(synth_features=.9, kind = "borderline2")],
#     ['SMOTE_regular', SubsetSMOTE(synth_features=.9, kind = "borderline2")]
# ]

samplers = [
    ['SMOTE_borderline1', SMOTE(kind = "borderline1", random_state=RANDOM_STATE)],
    ['SMOTE_borderline2', SMOTE(kind = "borderline2", random_state=RANDOM_STATE)],
    ['SMOTE_regular', SMOTE(random_state=RANDOM_STATE)],
    ['SubsetSMOTE', SubsetSMOTE(synth_features=.9)]
]


In [22]:
%%time
cv = StratifiedKFold(n_splits=5, random_state=42)
bass_mean_aucs = {}
for name, sampler in samplers:
    np.random.seed(100)
    print("working on", name)
    aucs = []
    for train, test in cv.split(X,y):
        clf = RandomForestClassifier(n_estimators=100, n_jobs=-2)
        bagged_subset_smote = SyntheticBaggingClassifier(synth_sampling=sampler,
                                                         base_estimator=clf,
                                                         n_estimators = 10,
                                                         ratio=.2,
                                                         replacement=False,
                                                         random_state=0)
        bagged_subset_smote.fit(X[train], y[train])
        y_scores = bagged_subset_smote.predict_proba(X[test])[:,1]
        aucs.append(roc_auc_score(y[test], y_scores))
    
    bass_mean_aucs[name] = aucs
   

working on SMOTE_borderline1
working on SMOTE_borderline2
working on SMOTE_regular
working on SubsetSMOTE
CPU times: user 3h 22min 19s, sys: 12.7 s, total: 3h 22min 32s
Wall time: 10min 44s


In [23]:
for key in bass_mean_aucs.keys():
    print(key)
    print(np.mean(bass_mean_aucs[key]), np.std(bass_mean_aucs[key]) )

SMOTE_borderline1
0.967758503838 0.021808726038
SMOTE_borderline2
0.977697396375 0.0128228665432
SMOTE_regular
0.979924740399 0.0105220968382
SubsetSMOTE
0.980179639199 0.0102272190444
